In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [4]:
df = pd.read_csv('../../data/part_one_annotation_50_input.csv')

In [5]:
df.columns

Index(['id', 'link', 'Q_difficulty', 'A_difficulty'], dtype='object')

In [6]:
# 'id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set',
#        'Q_Granular_breakdown'

In [7]:
q_df = df[['id', 'link', 'Q_difficulty']]
a_df = df[['id', 'link', 'A_difficulty']]

In [8]:
q_df.head()

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate


In [9]:
q_df

,id,link,Q_difficulty
0,77293473,https://stackoverflow.com/questions/77293473,Basic
1,77404924,https://stackoverflow.com/questions/77404924,Basic
2,77249095,https://stackoverflow.com/questions/77249095,Basic
3,77076663,https://stackoverflow.com/questions/77076663,Advanced
4,77019129,https://stackoverflow.com/questions/77019129,Intermediate
5,76992629,https://stackoverflow.com/questions/76992629,Intermediate
6,77370805,https://stackoverflow.com/questions/77370805,Intermediate
7,76784524,https://stackoverflow.com/questions/76784524,Basic
8,76608783,https://stackoverflow.com/questions/76608783,Intermediate
9,76697269,https://stackoverflow.com/questions/76697269,Intermediate


In [10]:
q_id_str = str([x for x in q_df['id']])[1:-1]
a_id_str = str([x for x in a_df['id']])[1:-1]

In [11]:
q_id_str

'77293473, 77404924, 77249095, 77076663, 77019129, 76992629, 77370805, 76784524, 76608783, 76697269, 76317584, 76570219, 76302571, 76548866, 76541602, 76264395, 76169010, 76169378, 76059076, 76045819, 76085427, 75779691, 75862649, 75637544, 75460677, 75454689, 75283999, 75279786, 75198212, 75057003, 75179892, 75239843, 74778102, 74774389, 74760874, 74751254, 74693871, 74844950, 74804354, 77553477, 77543317, 77511053, 77467788'

In [12]:
q_postypeid="'1'"
a_postypeid="'2'"



In [13]:
# q_sql = """select a.id 
#                 , 'Here is the question about software development. The title of the question is "'||a.title||'". and body of the question is "'||b.body||'". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'
#             from posts a 
#                , postsbody b
#             where a.id = b.id
#               and a.creationdate between '2021-11-30' and '2023-11-30' 
#               and a.posttypeid = {0} 
#               and a.tags like '%<python>%' 
#               and a.id in ({1})
# """

In [14]:
q_sql = """select a.id 
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [15]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [16]:
q_output = pd.DataFrame(rows, columns = [
  'id'
  ,'title'
  , 'body'
])


In [17]:
q_output.head()

,id,title,body
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett..."
1,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...
2,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...
3,74774389,How to join these 2 tables by date with ORM,<p>I have two querysets -</p>\n<pre><code>A = ...
4,74778102,Error when trying to save a form in django,<p>I am new to django and I am working on a pr...


In [18]:
q_output = pd.merge(q_output, q_df[['id', 'Q_difficulty']], on = ['id'] )

In [19]:
q_output['answer'] = 'Difficulty class : '+q_output['Q_difficulty'].map(str)

In [21]:
q_output['question'] = '<Title>'+q_output['title'].map(str)+'</Title>. <Question>'+q_output['body'].map(str)+'</Question>'


In [22]:
import re
q_output['question'] = q_output['question'].apply(lambda x:  re.sub(r";(?=\S)", "", x))
# .str.replace('";', "")

In [23]:
q_output[['id', 'question', 'answer']].to_csv('../../data/part_one_q_output_code_y.csv')

In [25]:
# q_output.iloc[[14, 25, 13, 21], :].to_csv('chk.csv')

In [26]:
q_output.head()

,id,title,body,Q_difficulty,answer,question
0,74693871,Why joblib is not recommended when saving kera...,"<p>According to <a href=""https://keras.io/gett...",Advanced,Difficulty class : Advanced,<Title>Why joblib is not recommended when savi...
1,74751254,Removing all duplicate images with different f...,<p>I am trying to iterate through a folder and...,Intermediate,Difficulty class : Intermediate,<Title>Removing all duplicate images with diff...
2,74760874,How do I extract a binary pattern from a 1D si...,<p>I need help for my school project. I am tak...,Intermediate,Difficulty class : Intermediate,<Title>How do I extract a binary pattern from ...
3,74774389,How to join these 2 tables by date with ORM,<p>I have two querysets -</p>\n<pre><code>A = ...,Intermediate,Difficulty class : Intermediate,<Title>How to join these 2 tables by date with...
4,74778102,Error when trying to save a form in django,<p>I am new to django and I am working on a pr...,Intermediate,Difficulty class : Intermediate,<Title>Error when trying to save a form in dja...
